# Part 3: Translation with Flan-T5

In this part you will experiment with Google Flan-T5 models for machine translation. The original T5 model was a unifed sequence-to-sequence encoder-decoder architecture pretrained on a variety of tasks including machine translation. The Flan line of models improved on the performance of the original T5 series.

In this part we will only apply the models, not train them. We will evaluate our results using the [Bleu score](https://en.wikipedia.org/wiki/BLEU), a common (but not perfect) quantitative metric for evaluating the quality of translations. Flan-T5 also comes in several different model sizes: We will study the impact of model size on performance by considering several different versions of Flan-T5.

**Learning objectives.** You will:
1. Examine an encoder-decoder sequence-to-sequence Flan-T5 transformer model
2. Apply Flan-T5 models to perform machine translation
3. Evaluate the quality of machine translations by computing Bleu scores with respect to reference translations
4. Study the affect

While it is possible to complete this assignment using CPU compute, it may be slow. To accelerate your training, consider using GPU resources such as `CUDA` through the CS department cluster. Alternatives include Google colab or local GPU resources for those running on machines with GPU support.

First, ensure that you have the `transformers` and `datasets` modules installed. We will use these modules for importing tokenizers, pretrained models, and datasets. You can run the following cells to try to install them with `pip` if needed. If you are using ondemand, ideally you would simply include `module load transformers` and `module load datasets` when making your initial reservation.

In [1]:
pip install transformers

Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


First we import the `flan-t5` tokenizer (shared across all model sizes) and demonstrate its characteristics and usage. Note that the API is the same as we saw previously for the `BERT` model -- may want to review the extra details in that earlier part.

Note that the example contains both English and French text.

(If you have trouble downloading the tokenizer, it is possible that you need to install the `sentencepiece` module, for example by `pip install sentencepiece`).

In [3]:
pip install SentencePiece

Note: you may need to restart the kernel to use updated packages.


In [4]:
# run but you do not need to modify this code

from transformers import T5Tokenizer

tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")

print("Vocabulary size: ", tokenizer.vocab_size)
tokenized = tokenizer(["The little black cat sleeps in the window",
                       "Le petit chat noir dort dans la fenêtre"], padding='longest')
print(tokenized)

tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Vocabulary size:  32000
{'input_ids': [[37, 385, 1001, 1712, 2085, 7, 16, 8, 2034, 1], [312, 4561, 3582, 9691, 5048, 247, 50, 25301, 1, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 0]]}


## Task 1

Below we import and preview the `flan-t5` model, beginning with the small version. You will also note that we are using 16-bit float representations to save on memory (this may be particularly relevant if you are using GPU compute for larger models, where the GPU may have limited memory available).

In [5]:
# run, but you do not need to modify this code
import torch
from transformers import T5ForConditionalGeneration

model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", torch_dtype=torch.float16)
print(model)

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/308M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=384, bias=False)
              (k): Linear(in_features=512, out_features=384, bias=False)
              (v): Linear(in_features=512, out_features=384, bias=False)
              (o): Linear(in_features=384, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 6)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseGatedActDense(
              (wi_0): Linear(in_features=512, out_features=1024, bias=False)
              (wi_1): Linear(in_features=512, out_features=1024, bias=False)
              (wo): 

Examine the `model.parameters()`. How much memory (in kilobytes (KB), megabytes (MB), or gigabytes (GB)) should it take to store the model itself, given the 16-bit (or 2 byte) precision specified in the import? Briefly explain.

In [6]:
# write code for task 1 here
total = sum(p.numel() for p in model.parameters())
print(f"# of model parameters: {total}")

# of model parameters: 76961152


*Briefly explain for task 1 here*

As shown above, there are 76,961,152 parameters in the model. Given that each parameter will take 2 bytes to store, the total memory required for storing this model is 76,961,152 * 2 bytes = 153,922,304 bytes, or about 153.92 MB.

## Task 2

Now we import and demonstrate the basic usage of the model `generate` method. This method autoregressively generates new text as we have discussed before in the context of causal language modeling, and supports different approaches (greedy, beam search, and sampling). The `generate` method API is [documented here](https://huggingface.co/docs/transformers/v4.46.0/en/main_classes/text_generation#transformers.GenerationMixin.generate).

The example below demonstrates encoding a *batch* of inputs and passing them to the model for autoregressive generation. The printed output is generated by the model for the first and second input in the batch respectively.

In [7]:
# run but you do not need to modify this code

input_text = ["The little black cat sleeps in the window", "The dog runs in the field"]
encoded = tokenizer(input_text, return_tensors="pt", padding="longest")

outputs = model.generate(**encoded, max_new_tokens=100)
print(outputs)

for out in outputs:
    print(tokenizer.decode(out, skip_special_tokens=True))

tensor([[   0,   37, 1712,   19,    3,    9, 1712,    5,    1],
        [   0,    3,   88,   19,    3,    9, 1782,    1,    0]])
The cat is a cat.
he is a dog


Note that the model did not translate the inputs. That is by design: Flan-T5 was pretrained on several different tasks including but not limited to machine translation.

In order to use the model for translation, we need to **prompt it** to do so, providing context (literally) connecting to its pretraining. There are several different promptings that should work, we demonstrate two below:

In [8]:
# run but you do not need to modify this code

input_text = ["The little black cat sleeps in the window", "The dog runs in the field"]
prompt = "Translate English into French: "
prompted_text = [prompt + in_text for in_text in input_text]
encoded = tokenizer(prompted_text, return_tensors="pt", padding="longest")

outputs = model.generate(**encoded, max_new_tokens=100)

for out in outputs:
    print(tokenizer.decode(out, skip_special_tokens=True))

La petite cat noir s'est en vertu de la fenêtre.
Le chien s'est en dehors de la terre.


For this task, your goal is use the model to translate a large collection of German text into English, drawing from a paired translation of the novel Jane Eyre. Below we download and prepare the data.

In [9]:
pip install datasets

Note: you may need to restart the kernel to use updated packages.


In [10]:
# run but you do not need to modify this code

from datasets import load_dataset
from torch.utils.data import Dataset

class OpusDataset(Dataset):
    def __init__(self, dataset_stream, num_examples):
        # Convert streaming dataset to list for random access
        self.examples = list(dataset_stream.take(num_examples))

    def __len__(self):
        return len(self.examples)

    def __getitem__(self, idx):
        return self.examples[idx]

# Load the dataset in streaming mode
dataset_stream = load_dataset(
    "opus_books",
    "de-en",
    split="train",
    streaming=True
)

# Create instance of custom dataset
dataset = OpusDataset(dataset_stream, num_examples=500)

# Print a few examples to verify
for i in range(100, 103):
    print(f"\nExample {i+1}:")
    print(f"German: {dataset[i]['translation']['de']}")
    print(f"English: {dataset[i]['translation']['en']}")
print(f"\nTotal examples loaded: {len(dataset)}")

README.md:   0%|          | 0.00/28.1k [00:00<?, ?B/s]


Example 101:
German: Dieser Vorwurf meiner Abhängigkeit war in meinen Ohren fast zum leeren, bedeutungslosen Singsang geworden, sehr schmerzlich und bedrückend, aber nur halb verständlich.
English: This reproach of my dependence had become a vague sing-song in my ear: very painful and crushing, but only half intelligible.

Example 102:
German: Nun fiel auch Miß Abbot ein: »Und Sie sollten auch nicht denken, daß Sie mit den Fräulein Reed und Mr. Reed auf gleicher Stufe stehen, weil Mrs. Reed Ihnen gütig erlaubt, mit ihren Kindern erzogen zu werden.
English: Miss Abbot joined in-- "And you ought not to think yourself on an equality with the Misses Reed and Master Reed, because Missis kindly allows you to be brought up with them.

Example 103:
German: Diese werden einmal ein großes Vermögen haben, und Sie sind arm. Sie müssen demütig und bescheiden sein und versuchen, sich den andern angenehm zu machen.«
English: They will have a great deal of money, and you will have none: it is your pl

**Use the Flan-T5 model to translate all of the German text in the dataset into English.** Even with GPU compute, this may take several minutes, but should not take hours. We encourage you to add some output every 10 or 50 examples so that you can track the progress, though you are not required to do so.

**Select at least three examples from the dataset and print the model translation as well as the real English text.**

In [11]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

prompt = "Translate German into English: "
predicted = []

for i in range(len(dataset)):
    german_text = dataset[i]['translation']['de']
    prompted_text = [prompt + german_text]
    encoded = tokenizer(prompted_text, return_tensors="pt", padding="longest").to(device)
    output = model.generate(**encoded, max_new_tokens=100)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted.append(pred)
    if (i + 1) % 10 == 0:
        print(f"\nExample {i+1}:")
        print(f"German: {german_text}")
        print(f"Prediction: {pred}")

print("\nSample Model Translations vs. Ground Truth:\n")
for i in [100, 200, 300]:
    print(f"Example {i+1}:")
    print(f"German: {dataset[i]['translation']['de']}")
    print(f"Actual English: {dataset[i]['translation']['en']}")
    print(f"Model Translation: {predicted[i]}\n")



Example 10:
German: »Was sagt denn Bessie, daß ich gethan habe?« fragte ich.
Prediction: "How is Bessie saying that I have been?" I asked.

Example 20:
German: Es waren jene, die von den Verstecken der Seevögel handelten, von jenen einsamen Felsen und Klippen, welche nur sie allein bewohnen, von der Küste Norwegens, die von ihrer äußersten südlichen Spitze, dem Lindesnäs bis zum Nordkap mit Inseln besäet ist. Wo der nördliche Ozean, in wildem Wirbel Um die nackten, öden Inseln tobt Des ultima Thule; und das atlantische Meer Sich stürmisch zwischen die Hebriden wälzt.
Prediction: It was those who were surrounded by the sand and sand that only they could afford, from the Norwegen coast, which is a sand-covered island, which is a sand-covered island, which is a sand-covered island, and the sand-covered island is a sand-covered island, and the sand-covered island is a sand-covered island, which is

Example 30:
German: »Bah, Frau Träumerin!« ertönte John Reeds Stimme; dann hielt er inne; a

## Task 3

Translations are difficult to evaluate quantitatively and without expert human translators. One common metric is the [BLEU score](https://en.wikipedia.org/wiki/BLEU).

The below example demonstrates calculating BLEU scores with the `evaluate` module from Hugging Face. You can [see the documentation here](https://huggingface.co/spaces/evaluate-metric/bleu). The first value in the results dictionary gives the score. Normally the score is reported on a scale from 0-100; this implementation reports it on a 0-1 scale. Higher values are better, but scores of 1 are not necessarily expected given the many possible ways to translate.

Note that `predictions` is a list of strings, but `references` is a list of lists of strings. This is because a single predicted translation could potentially have multiple equally good reference translations. In our case however we just have the single translation per pair, so `references` will be a list of lists, each with a single element.

In [12]:
# run but you do not need to modify this code

import evaluate

predictions = ["the black cat is sleeping in the sun by the window",
               "the dog runs in the field while it rains"]
references = [["the black cat is sleeps on the sun by the window"],
              ["the dog run in the field while it rain"]]

bleu = evaluate.load("bleu")
results = bleu.compute(predictions=predictions, references=references)
print(results)

ModuleNotFoundError: No module named 'evaluate'

**Calculate the `BLEU` score of your translations from task 2 against the real English text`. Report your results.**

In [ ]:
# write code for task 3 here
refs = [dataset[i]['translation']['de'] for i in range(len(dataset))]
scores = bleu.compute(predictions=predicted, references=refs)
print(scores)

{'bleu': 0.02636236611533724, 'precisions': [0.1488839987425338, 0.031028886725089258, 0.015632338141563232, 0.007285089849441476], 'brevity_penalty': 0.9788499415278665, 'length_ratio': 0.9790704832256079, 'translation_length': 15905, 'reference_length': 16245}


## Task 4



In this task, study the impact of model scale on the quality of the resulting translations. Earlier we used `model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-small", torch_dtype=torch.float16)` to import a `flan-t5-small` model.

**Use two additional models: `flan-t5-base` and `flan-t5-large` to generate translations of the same dataset. Evaluate and report the BLEU score of both translations.**

In [ ]:
# write code for task 4 here
model_base = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base", torch_dtype=torch.float16)
model_base.to(device)
predicted_base = []
for i in range(len(dataset)):
    german_text = dataset[i]['translation']['de']
    prompted_text = [prompt + german_text]
    encoded = tokenizer(prompted_text, return_tensors="pt", padding="longest").to(device)
    output = model_base.generate(**encoded, max_new_tokens=100)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_base.append(pred)
    # if (i + 1) % 50 == 0:
    #     print(f"\nExample {i+1}:")
    #     print(f"German: {german_text}")
    #     print(f"Prediction: {pred}")

print("\nSample Model Translations vs. Ground Truth:\n")
for i in [100, 200, 300]:
    print(f"Example {i+1}:")
    print(f"German: {dataset[i]['translation']['de']}")
    print(f"Actual English: {dataset[i]['translation']['en']}")
    print(f"Model Translation: {predicted_base[i]}\n")


model_large = T5ForConditionalGeneration.from_pretrained("google/flan-t5-large", torch_dtype=torch.float16)
model_large.to(device)
predicted_large = []
for i in range(len(dataset)):
    german_text = dataset[i]['translation']['de']
    prompted_text = [prompt + german_text]
    encoded = tokenizer(prompted_text, return_tensors="pt", padding="longest").to(device)
    output = model_large.generate(**encoded, max_new_tokens=100)
    pred = tokenizer.decode(output[0], skip_special_tokens=True)
    predicted_large.append(pred)
    # if (i + 1) % 50 == 0:
    #     print(f"\nExample {i+1}:")
    #     print(f"German: {german_text}")
    #     print(f"Prediction: {pred}")

print("\nSample Model Translations vs. Ground Truth:\n")
for i in [100, 200, 300]:
    print(f"Example {i+1}:")
    print(f"German: {dataset[i]['translation']['de']}")
    print(f"Actual English: {dataset[i]['translation']['en']}")
    print(f"Model Translation: {predicted_large[i]}\n")

scores_base = bleu.compute(predictions=predicted_base, references=refs)
print(f"Scores - base: {scores_base}")
scores_large = bleu.compute(predictions=predicted_large, references=refs)
print(f"Scores - large: {scores_large}")


Sample Model Translations vs. Ground Truth:

Example 101:
German: Dieser Vorwurf meiner Abhängigkeit war in meinen Ohren fast zum leeren, bedeutungslosen Singsang geworden, sehr schmerzlich und bedrückend, aber nur halb verständlich.
Actual English: This reproach of my dependence had become a vague sing-song in my ear: very painful and crushing, but only half intelligible.
Model Translation: This draft of my own was quickly becoming a painful and painful singing, very painful and painful, but only half a logical one.

Example 201:
German: Welche seltene Höflichkeit!
Actual English: Wonderful civility this!
Model Translation: What rare happiness!

Example 301:
German: »Ich weiß es nicht.
Actual English: "I don't know.
Model Translation: "I don't know.


Sample Model Translations vs. Ground Truth:

Example 101:
German: Dieser Vorwurf meiner Abhängigkeit war in meinen Ohren fast zum leeren, bedeutungslosen Singsang geworden, sehr schmerzlich und bedrückend, aber nur halb verständlich.
Ac